In [1]:
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
pd.set_option("display.max_rows", 201)

# Scraping

This is a nice wrapper for getting the soup object from a url. 

In [2]:
def get_soup(url):
    """
    Small wrapper around urllib and BeautifulSoup. Takes a url
    and produces the soup object. Includes a header so that 
    http errors are less likely.
    """
    hdr = {"User-Agent": "Mozilla/5.0"}
    req = Request(url, headers=hdr)
    page = urlopen(req)
    soup = BeautifulSoup(page, "html")
    return soup


def combine_to_df(
    artist_list, album_list, rank_list, url, year_list=None, label_list=None
):
    df = pd.DataFrame({"artist": artist_list, "album": album_list, "rank": rank_list,})

    if year_list:
        df["year"] = year_list

    if label_list:
        df["label"] = label_list

    df["reviewer_url"] = url

    return df

# Pitchfork

In [3]:
url = (
    "https://pitchfork.com/features/lists-and-guides/the-200-best-albums-of-the-2010s/"
)


soup = get_soup(url)

In [4]:
artist = []
album = []
year = []
for tag in soup.find_all("h2"):
    artist.append(tag.contents[0][:-2])
    album.append(tag.contents[1].contents[0])
    year.append(tag.contents[2][2:-1])

# cleaning
year[117] = "2013"
year[135] = "2015"

pitchfork_df = combine_to_df(
    artist_list=artist,
    album_list=album,
    rank_list=[200 - n for n in range(200)],
    year_list=year,
    url=url,
)

In [5]:
pitchfork_df.head()

,artist,album,rank,year,reviewer_url
0,Ratking,So It Goes,200,2014,https://pitchfork.com/features/lists-and-guide...
1,Wu Lyf,Go Tell Fire to the Mountain,199,2011,https://pitchfork.com/features/lists-and-guide...
2,Jean Grae / Quelle Chris,Everything’s Fine,198,2018,https://pitchfork.com/features/lists-and-guide...
3,Fatima Al Qadiri,Genre-Specific Xperience,197,2011,https://pitchfork.com/features/lists-and-guide...
4,Portal,Vexovoid,196,2013,https://pitchfork.com/features/lists-and-guide...


# billboard

In [6]:
url = "https://www.billboard.com/articles/news/list/8543722/best-albums-of-the-2010s-top-100"

soup = get_soup(url)

In [7]:
billboard_rank = []
artist = []
album = []
year = []

for tag in soup.find_all("strong"):
    billboard_rank.append(tag.text.split(".")[0])
    artist.append(tag.text.split(". ", 1)[1].split(",", 1)[0])
    album.append(tag.text.split(". ", 1)[1].split(",", 1)[1].split("(")[0])
    year.append(tag.text.split(". ", 1)[1].split(",", 1)[1].split("(")[1][:-1])

billboard_df = combine_to_df(
    album_list=album,
    artist_list=artist,
    rank_list=billboard_rank,
    year_list=year,
    url=url,
)

In [8]:
# Small tweaks
billboard_df.iloc[0, 3] = "2018"
billboard_df.iloc[44, 0] = "Tyler, the Creator"
billboard_df.iloc[44, 1] = "IGOR"

In [9]:
billboard_df.head()

,artist,album,rank,year,reviewer_url
0,Lady Gaga & Bradley Cooper,A Star Is Born Soundtrack,100,2018,https://www.billboard.com/articles/news/list/8...
1,Lady Antebellum,Need You Now,99,2010,https://www.billboard.com/articles/news/list/8...
2,Japandroids,Celebration Rock,98,2012,https://www.billboard.com/articles/news/list/8...
3,Porter Robinson,Worlds,97,2014,https://www.billboard.com/articles/news/list/8...
4,Ed Sheeran,x,96,2014,https://www.billboard.com/articles/news/list/8...


# Stereogum

In [10]:
url = "https://www.stereogum.com/featured/best-albums-of-the-2010s-list/"

soup = get_soup(url)

In [11]:
def h2_if_nonempty(tag):
    if tag.contents:
        if "h2" in tag.name:
            return True
    else:
        return False

In [12]:
stereogum_rank = []
artist = []
album = []
label = []
year = []
for tag in soup.find_all(h2_if_nonempty):
    stereogum_rank.append(tag.contents[0].contents[0])
    artist.append(tag.contents[1])
    album.append(tag.contents[2].contents[0])
    label.append(tag.contents[4].contents[0].split(",")[0])
    year.append(tag.contents[4].contents[0].split(",")[1])

# Cleaning
artist = [x[1:-3] for x in artist]
label = [x[1:] for x in label]
year = [x[1:-1] for x in year]

stereogum_df = combine_to_df(
    album_list=album,
    artist_list=artist,
    rank_list=stereogum_rank,
    year_list=year,
    url=url,
)

In [13]:
stereogum_df.head()

,artist,album,rank,year,reviewer_url
0,Future,Pluto,100,2012,https://www.stereogum.com/featured/best-albums...
1,Car Seat Headrest,Teens Of Denial,99,2016,https://www.stereogum.com/featured/best-albums...
2,PUP,The Dream Is Over,98,2016,https://www.stereogum.com/featured/best-albums...
3,Courtney Barnett,"Sometimes I Sit And Think, And Sometimes I Jus...",97,2015,https://www.stereogum.com/featured/best-albums...
4,Colleen Green,I Want To Grow Up,96,2015,https://www.stereogum.com/featured/best-albums...


# Consequence of Sound 

In [14]:
url =  "https://consequenceofsound.net/2019/11/top-albums-of-the-2010s/full-post/"

soup = get_soup(url)

In [15]:
cos_rank = []
artist = []
album = []
year = []

for tag in soup.find_all('h2')[2:-1]:
    cos_rank.append(tag.contents[0].split('.')[0])
    artist.append(tag.contents[0].split('.', 1)[1][1:-3])
    album.append(tag.contents[1].contents[0])
    year.append(tag.contents[2][2:-1])

In [48]:
cos_df = combine_to_df(
    album_list=album,
    artist_list=artist,
    rank_list=cos_rank,
    year_list=year,
    url=url,
)

# fixes
cos_df.iloc[67, 3] = '2015'
cos_df.iloc[83, 3] = '2016'
cos_df.iloc[89, 3] = '2012'

In [49]:
cos_df

,artist,album,rank,year,reviewer_url
0,Justin Bieber,Purpose,100,2011,https://genius.com/a/the-genius-communitys-100...
1,Katy Perry,Teenage Dream,99,2017,https://genius.com/a/the-genius-communitys-100...
2,Bon Iver,"Bon Iver, Bon Iver",98,2016,https://genius.com/a/the-genius-communitys-100...
3,Billie Eilish,"WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?",97,2013,https://genius.com/a/the-genius-communitys-100...
4,Schoolboy Q,Oxymoron,96,2017,https://genius.com/a/the-genius-communitys-100...
5,Ariana Grande,Dangerous Woman,95,2010,https://genius.com/a/the-genius-communitys-100...
6,Isaiah Rashad,Cilvia Demo,94,2013,https://genius.com/a/the-genius-communitys-100...
7,alt-j,An Awesome Wave,93,2013,https://genius.com/a/the-genius-communitys-100...
8,Flying Lotus,Cosmogramma,92,2018,https://genius.com/a/the-genius-communitys-100...
9,Logic,Under Pressure,91,2013,https://genius.com/a/the-genius-communitys-100...


# Time

In [18]:
url =  "https://time.com/5725768/best-albums-2010s-decade/"

soup = get_soup(url)

In [19]:
tag_list = [tag for tag in soup.find_all("strong")]
tag_list.remove(tag_list[-1])

artist = []
album = []
year = []

for tag in tag_list:
    artist.append(tag.text.split(',')[0].strip())
    album.append(tag.text.split(',')[1].split('(')[0].strip())
    year.append(tag.text.split(',')[1].split('(')[1].strip()[:-1])

In [20]:
time_df = combine_to_df(
    album_list=album,
    artist_list=artist,
    rank_list=10,
    year_list=year,
    url=url,
)

In [21]:
time_df

,artist,album,rank,year,reviewer_url
0,Fiona Apple,The Idler Wheel…,10,2012,https://time.com/5725768/best-albums-2010s-dec...
1,Miguel,Kaleidoscope Dream,10,2012,https://time.com/5725768/best-albums-2010s-dec...
2,Beauty Pill,Beauty Pill Describes Things As They Are,10,2015,https://time.com/5725768/best-albums-2010s-dec...
3,Carly Rae Jepsen,E•MO•TION,10,2015,https://time.com/5725768/best-albums-2010s-dec...
4,Beyoncé,Lemonade,10,2016,https://time.com/5725768/best-albums-2010s-dec...
5,Leonard Cohen,You Want It Darker,10,2016,https://time.com/5725768/best-albums-2010s-dec...
6,Miranda Lambert,The Weight of These Wings,10,2016,https://time.com/5725768/best-albums-2010s-dec...
7,Solange,A Seat At the Table,10,2016,https://time.com/5725768/best-albums-2010s-dec...
8,Kendrick Lamar,DAMN.,10,2017,https://time.com/5725768/best-albums-2010s-dec...
9,Ozuna,Aura,10,2018,https://time.com/5725768/best-albums-2010s-dec...


# Paste Magazine


In [22]:
urls = [
    "https://www.pastemagazine.com/articles/2019/10/best-albums-of-the-2010s-paste.html",
    "https://www.pastemagazine.com/articles/2019/10/best-albums-of-the-2010s-paste.html?p=2",
    "https://www.pastemagazine.com/articles/2019/10/best-albums-of-the-2010s-paste.html?p=3",
    "https://www.pastemagazine.com/articles/2019/10/best-albums-of-the-2010s-paste.html?p=4",
]

In [23]:
paste_rank = []
artist = []
album = []
year = []

tag_list = []
for url in urls:
    soup = get_soup(url)

    for tag in soup.find_all("b", class_="big"):
        tag_list.append(tag)

tag_list = tag_list[1:]

for tag in tag_list:
    paste_rank.append(tag.text.split(".", 1)[0])
    artist.append(tag.text.split(".", 1)[1].split(":", 1)[0][1:])
    album.append(tag.text.split(":", 1)[1][1:-7])
    year.append(tag.text[-5:-1])

paste_df = combine_to_df(
    album_list=album,
    artist_list=artist,
    rank_list=paste_rank,
    year_list=year,
    url=urls[0],
)

In [24]:
paste_df.head()

,artist,album,rank,year,reviewer_url
0,Drake,Take Care,100,2011,https://www.pastemagazine.com/articles/2019/10...
1,Japanese Breakfast,Soft Sounds from Another Planet,99,2017,https://www.pastemagazine.com/articles/2019/10...
2,Nick Cave & The Bad Seeds,Skeleton Tree,98,2016,https://www.pastemagazine.com/articles/2019/10...
3,Deafheaven,Sunbather,97,2013,https://www.pastemagazine.com/articles/2019/10...
4,"Tyler, The Creator",Flower Boy,96,2017,https://www.pastemagazine.com/articles/2019/10...


# Genius

In [25]:
url = "https://genius.com/a/the-genius-communitys-100-best-albums-of-the-2010s"

soup = get_soup(url)

In [26]:
genius_rank = []
artist = []
album = []

for tag in soup.find_all("div", class_="g_list-item-header"):
    genius_rank.append(tag.text.replace("\n", "")[10:].split(" ", 1)[0])
    artist.append(tag.text.replace("\n", "")[10:].split(" ", 1)[1][17:].split(" – ")[0])
    album.append(tag.text.replace("\n", "")[10:].split(" ", 1)[1][17:].split(" – ")[1])

year_list = [
    '2015'
]

In [27]:
genius_df = combine_to_df(
    album_list=album,
    artist_list=artist,
    rank_list=genius_rank,
    url=url,
)

In [84]:
genius_df

,artist,album,rank,reviewer_url
0,Justin Bieber,Purpose,100,https://genius.com/a/the-genius-communitys-100...
1,Katy Perry,Teenage Dream,99,https://genius.com/a/the-genius-communitys-100...
2,Bon Iver,"Bon Iver, Bon Iver",98,https://genius.com/a/the-genius-communitys-100...
3,Billie Eilish,"WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?",97,https://genius.com/a/the-genius-communitys-100...
4,Schoolboy Q,Oxymoron,96,https://genius.com/a/the-genius-communitys-100...
5,Ariana Grande,Dangerous Woman,95,https://genius.com/a/the-genius-communitys-100...
6,Isaiah Rashad,Cilvia Demo,94,https://genius.com/a/the-genius-communitys-100...
7,alt-j,An Awesome Wave,93,https://genius.com/a/the-genius-communitys-100...
8,Flying Lotus,Cosmogramma,92,https://genius.com/a/the-genius-communitys-100...
9,Logic,Under Pressure,91,https://genius.com/a/the-genius-communitys-100...


# Combining Data 

In [72]:
def reformat(df_list):
    """
    Trims any external whitespaces on all strings and turns rank and year into integers.
    """
    for df in df_list:

        # trim any whitespace around the strings in each column
        for column in df.columns:
            if df[column].dtype == 'object':
                df[column] = df[column].apply(lambda x: x.strip())

        # Make years and ranks integers
        
        if "year" in df.columns:
            df["year"] = df["year"].astype('Int64')
            
        df["rank"] = df["rank"].astype(int)

    return

In [170]:
df_list = [
    pitchfork_df,
    billboard_df,
    stereogum_df,
    cos_df,
    time_df,
    paste_df,
    genius_df
]

reformat(df_list)

In [171]:
albums = pd.concat(df_list, sort=False)

In [172]:
albums['year'] = albums['year'].astype('Int64')
albums.reset_index()

,index,artist,album,rank,year,reviewer_url
0,0,Ratking,So It Goes,200,2014,https://pitchfork.com/features/lists-and-guide...
1,1,Wu Lyf,Go Tell Fire to the Mountain,199,2011,https://pitchfork.com/features/lists-and-guide...
2,2,Jean Grae / Quelle Chris,Everything’s Fine,198,2018,https://pitchfork.com/features/lists-and-guide...
3,3,Fatima Al Qadiri,Genre-Specific Xperience,197,2011,https://pitchfork.com/features/lists-and-guide...
4,4,Portal,Vexovoid,196,2013,https://pitchfork.com/features/lists-and-guide...
...,...,...,...,...,...,...
702,95,Drake,Take Care,5,NaN,https://genius.com/a/the-genius-communitys-100...
703,96,Beyoncé,Lemonade,4,NaN,https://genius.com/a/the-genius-communitys-100...
704,97,Frank Ocean,Blonde,3,NaN,https://genius.com/a/the-genius-communitys-100...
705,98,Kendrick Lamar,To Pimp A Butterfly,2,NaN,https://genius.com/a/the-genius-communitys-100...


## Fixing missing years

In [173]:
def get_year(album, df_list=df_list):

    # ideally a singleton
    
    years = [] 
    for df in df_list:
        found_year_in_df = False 
        
        if album in df["album"].values:
#             if artist in df["artist"].values:
#                 print("found artist and album")
            if "year" in df.columns:
                found_year_in_df = True
#             else:
#                 print("found album but not artist")
#         else:
#             if artist in df["artist"].values:
# #                 print("found artist but no album")
#             else:
# #                 print("found neither")

        if found_year_in_df:
            years.append(
                df.loc[df["album"] == album, "year",].values[0]
            )
    return years

In [184]:
# Make sure that each album's year is correctly recorded.

# first, gather the subset of albums with an incorrect listing.
weird_years = (albums.album.apply(get_year).apply(set).apply(len) != 1)
albums['year_list'] = albums['album'].apply(get_year)

def most_common(lst):
    """ Taken from 
    https://stackoverflow.com/questions/1518522/find-the-most-common-element-in-a-list
    """
    return max(set(lst), key=lst.count)

# Overwrite the year with the most common year from the year list.
# This may not be the best in all scenarios, but it is what I went with.
albums.loc[weird_years, "year"] = albums.loc[weird_years, "year_list"].apply(most_common)
albums.drop("year_list", axis = 1, inplace=True)

Next, use a fuzzyfinder tool to make sure no albums appear more than once.

# Data Exploration

In [83]:
# Do a word bubble of this
albums['artist'].unique()

array(['Ratking', 'Wu Lyf', 'Jean Grae / Quelle Chris',
       'Fatima Al Qadiri', 'Portal', 'Downtown Boys', 'Titus Andronicus',
       'Lil Peep', 'Kelela', 'Kate Bush', 'Huerco S.', 'Miranda Lambert',
       'Pusha-T', 'Iceage', 'Various Artists', 'Hailu Mergia',
       'Soccer Mommy', 'Elysia Crampton', 'G.L.O.S.S.', 'Jamila Woods',
       'Chvrches', 'Shabazz Palaces', 'Savages', 'Margo Price',
       'Gil Scott-Heron', 'Jessie Ware', 'Sharon Van Etten',
       'Hurray for the Riff Raff', 'Popcaan', 'JPEGMAFIA', 'Skee Mask',
       'Paramore', 'Julianna Barwick', '(Sandy) Alex G', 'Nicki Minaj',
       'Jay Som', 'Hop Along', 'Meek Mill', 'Sturgill Simpson',
       'The 1975', 'Noname', 'Pallbearer', 'Sleigh Bells', 'Jenny Lewis',
       '21 Savage / Metro Boomin', 'Mica Levi', 'Purple Mountains',
       'Arctic Monkeys', 'Lady Gaga', 'Playboi Carti', 'Mac DeMarco',
       'Nicolas Jaar', 'The National', 'Sheer Mag', 'Chief Keef',
       'PJ Harvey', 'Weyes Blood', 'Fennesz', 'Ric

Make a new column for a score conversion.